### Imports

In [5]:
import json
import pandas as pd
import numpy as np
import glob
from tqdm.auto import tqdm

In [6]:
%%time
with open('../../data/ru_reveal_text_changes_final_ner.json') as user_file:
    ru_reveal_text_changes_ner = json.load(user_file)
print("Number of samples:", len(ru_reveal_text_changes_ner))

Number of samples: 18514
CPU times: user 561 ms, sys: 80.1 ms, total: 641 ms
Wall time: 651 ms


In [7]:
np.random.choice(ru_reveal_text_changes_ner, size=5)

array([{'page_name': 'Аникеев, Андрей Анатольевич', 'ne_added': [{'text': 'Соединенных Штатов Америки', 'label': 'LOC', 'lemma': 'соединить штат америка'}], 'ne_removed': [{'text': 'Соединенных Штатов Америки', 'label': 'LOC', 'lemma': 'соединить штат америка'}, {'text': 'Кремля', 'label': 'LOC', 'lemma': 'кремль'}, {'text': 'Украины', 'label': 'LOC', 'lemma': 'украина'}, {'text': 'Украины', 'label': 'LOC', 'lemma': 'украина'}, {'text': 'ЕС', 'label': 'LOC', 'lemma': 'ес'}, {'text': 'Великобритании', 'label': 'LOC', 'lemma': 'великобритания'}, {'text': 'США', 'label': 'LOC', 'lemma': 'сша'}, {'text': 'Канады', 'label': 'LOC', 'lemma': 'канада'}, {'text': 'Швейцарии', 'label': 'LOC', 'lemma': 'швейцария'}, {'text': 'Австралии', 'label': 'LOC', 'lemma': 'австралия'}, {'text': 'Японии', 'label': 'LOC', 'lemma': 'япония'}, {'text': 'Украины', 'label': 'LOC', 'lemma': 'украина'}, {'text': 'Новой Зеландии', 'label': 'LOC', 'lemma': 'новый зеландия'}], 'ne_changed': [[[{'text': 'Аникеева', 'l

### Consts

In [8]:
import joblib

path = '../../data/ru_wiki_final_dataset.data'
ru_reveal_text_changes = joblib.load(path)

In [9]:
i = 0
for item in ru_reveal_text_changes:
    if item["status"] == 4:
        i += 1
i

18514

In [10]:
TOTAL_CHANGED_ARTICLES = i

### Summary Statistics

In [11]:
len(ru_reveal_text_changes_ner)

18514

In [12]:
i = 0
for item in ru_reveal_text_changes_ner:
    if not item["ne_added"]:
        i += 1
print(f"There are {i} empty list of added NE. It is {round(i / len(ru_reveal_text_changes_ner) * 100, 2)}%")

There are 10608 empty list of added NE. It is 57.3%


In [13]:
i = 0
for item in ru_reveal_text_changes_ner:
    if not item["ne_removed"]:
        i += 1
print(f"There are {i} empty list of removed NE. It is {round(i / len(ru_reveal_text_changes_ner) * 100, 2)}%")

There are 9045 empty list of removed NE. It is 48.85%


In [14]:
i = 0
for item in ru_reveal_text_changes_ner:
    if not item["ne_changed"]:
        i += 1
print(f"There are {i} empty list of changed NE. It is {round(i / len(ru_reveal_text_changes_ner) * 100, 2)}%")

There are 4038 empty list of changed NE. It is 21.81%


### Final statistics

In [30]:
wiki = []

# for page in np.random.choice(ru_reveal_text_changes_ner, size=10):
for page in tqdm(ru_reveal_text_changes_ner):
    # print(page, "\n")
    
    # extract ne_added
    ne_added = []
    if page["ne_added"]:
        for item in page["ne_added"]:
            ne_added.append(item)
        
    # extract ne_removed
    ne_removed = []
    if page["ne_removed"]:
        for item in page["ne_removed"]:
            ne_removed.append(item)
                
    # extract ne_changed_0 and ne_changed_1
    ne_changed_0, ne_changed_1 = [], []
    if page["ne_changed"]:
        for items in page["ne_changed"]:
            ne_changed_0.extend(items[0])
            ne_changed_1.extend(items[1])
    
    ne_wikipedia = ne_removed + ne_changed_0
    ne_ruwiki = ne_added + ne_changed_1
    
    if ne_wikipedia:
        ne_wikipedia_df = pd.DataFrame(ne_wikipedia)[["lemma", "label"]].drop_duplicates()
    else:
        ne_wikipedia_df = pd.DataFrame(columns=["lemma", "label"])
    
    if ne_ruwiki:
        ne_ruwiki_df = pd.DataFrame(ne_ruwiki)[["lemma", "label"]].drop_duplicates()
    else:
        ne_ruwiki_df = pd.DataFrame(columns=["lemma", "label"])
    
    removed = set(ne_wikipedia_df["lemma"].to_list()) - set(ne_ruwiki_df["lemma"].to_list())
    # print("Removed:", removed)
    
    added = set(ne_ruwiki_df["lemma"].to_list()) - set(ne_wikipedia_df["lemma"].to_list())
    # print("Added", added)
    
    page_results = {
        "page_name": page["page_name"],
        "removed": ne_wikipedia_df.loc[ne_wikipedia_df["lemma"].isin(list(removed))].to_dict('records'),
        "added": ne_ruwiki_df.loc[ne_ruwiki_df["lemma"].isin(list(added))].to_dict('records')
    }
    
    # print(page_results)
    # print("=" * 100)
    
    wiki.append(page_results)

  0%|          | 0/18514 [00:00<?, ?it/s]

In [31]:
len(wiki)

18514

In [32]:
ner_removed = []
for page in wiki:
    for item in page["removed"]:
        ner_removed.append({
            "page_name": page["page_name"],
            "lemma": item["lemma"],
            "label": item["label"],
        })
len(ner_removed)

72146

In [33]:
ner_added = []
for page in wiki:
    for item in page["added"]:
        ner_added.append({
            "page_name": page["page_name"],
            "lemma": item["lemma"],
            "label": item["label"],
        })
len(ner_added)

41188

In [34]:
ner_removed_df = pd.DataFrame(ner_removed)
ner_added_df = pd.DataFrame(ner_added)

In [35]:
ner_removed_df.head()

,page_name,lemma,label
0,Железнодорожный (Нижегородская область),сбербанк,ORG
1,Улица Чкалова (Мелитополь),вулиця чкалова,PER
2,"Чтак, Валерий Сергеевич","галерея ""triangle""",ORG
3,"Чтак, Валерий Сергеевич",галерея triangle,ORG
4,"Чтак, Валерий Сергеевич",triangle,ORG


In [36]:
ner_added_df.head()

,page_name,lemma,label
0,"Чтак, Валерий Сергеевич",triangle gallery,ORG
1,Мирное (Славянский район),государственный комитет по земельным ресурс,ORG
2,Мирное (Славянский район),днр,LOC
3,Мирное (Славянский район),донецкий народный республика,LOC
4,Мирное (Славянский район),российский федерация,LOC


In [37]:
pd.DataFrame(ner_removed_df[["lemma", "label"]].value_counts()[:10] / TOTAL_CHANGED_ARTICLES * 100)

,,0
lemma,label,
россия,LOC,22.518094
украина,LOC,21.367614
верховный рада,ORG,12.736308
донецкий область,LOC,4.493897
рф,LOC,4.488495
автономный республика крым,LOC,4.385870
крым,LOC,3.894350
луганский область,LOC,3.348817
сша,LOC,2.668251


In [38]:
pd.DataFrame(ner_removed_df[["lemma", "label"]].value_counts()[:10])

,,0
lemma,label,
россия,LOC,4169
украина,LOC,3956
верховный рада,ORG,2358
донецкий область,LOC,832
рф,LOC,831
автономный республика крым,LOC,812
крым,LOC,721
луганский область,LOC,620
сша,LOC,494


In [39]:
pd.DataFrame(ner_added_df[["lemma", "label"]].value_counts()[:10] / TOTAL_CHANGED_ARTICLES * 100)

,,0
lemma,label,
российский федерация,LOC,22.291239
донецкий народный республика,LOC,8.631306
днр,LOC,6.000864
луганский народный республика,LOC,5.941450
украина,LOC,5.768608
лнр,LOC,4.747758
россия,LOC,3.375824
республика крым,LOC,1.804040
рф,LOC,0.950632


In [40]:
pd.DataFrame(ner_added_df[["lemma", "label"]].value_counts()[:10])

,,0
lemma,label,
российский федерация,LOC,4127
донецкий народный республика,LOC,1598
днр,LOC,1111
луганский народный республика,LOC,1100
украина,LOC,1068
лнр,LOC,879
россия,LOC,625
республика крым,LOC,334
рф,LOC,176
